In [1]:
import os
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import image
import csv
import shutil
from tqdm import tqdm
from PIL import Image
os.getcwd()
print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.2.0


In [7]:
#Ref: https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c

from keras.models import Sequential
#Import from keras_preprocessing not from keras.preprocessing
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from keras.optimizers import SGD
import pandas as pd
import numpy as np
import keras 
from keras.layers import *
from keras.models import *

traindf=pd.read_csv('train.csv',dtype=str)
testdf=pd.read_csv('test.csv',dtype=str)

#traindf['age']=traindf.age.astype('int64')

#testdf['age']=traindf.age.astype('int64')

traindf.drop(['age'],axis=1,inplace=True)
testdf.drop(['age'],axis=1,inplace=True)

print(traindf.head())
print(testdf.head())

print(traindf.info())
print(testdf.info())


                                           full_path gender
0  data/imdb/50/nm0004950_rm4036071168_1972-4-17_...      0
1  data/imdb/77/nm0004977_rm474265344_1973-7-15_2...      1
2  data/imdb/21/nm0019221_rm3640760320_1948-9-11_...      1
3  data/imdb/34/nm1289434_rm2872508160_1983-2-23_...      0
4  data/imdb/90/nm0001590_rm687003392_1952-5-15_2...      1
                                           full_path gender
0  data/imdb/44/nm0089244_rm238668800_1939-8-7_19...      0
1          data/wiki/21/32518421_1991-09-01_2015.jpg      1
2          data/wiki/76/38687576_1939-08-29_1961.jpg      1
3  data/imdb/88/nm0005388_rm594123264_1970-5-15_2...      1
4  data/imdb/25/nm0001325_rm2254805248_1959-11-3_...      1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156918 entries, 0 to 156917
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   full_path  156918 non-null  object
 1   gender     156918 non-null  object
dtypes: ob

In [8]:
test_labels = testdf['gender']
print(test_labels)

0        0
1        1
2        1
3        1
4        1
        ..
52561    1
52562    1
52563    0
52564    1
52565    1
Name: gender, Length: 52566, dtype: object


In [9]:
#test_images = []

#for filename in testdf.full_path: 
#    im=image.imread(filename)
#    test_images.append(im)

In [10]:
#len(test_images)

In [13]:
train, valid = train_test_split(traindf, test_size=0.2, random_state=42, shuffle=True)
print(train.shape)
print(valid.shape)

(125534, 2)
(31384, 2)


In [22]:
valid.head()

,full_path,gender
13281,data/imdb/69/nm0006969_rm3409875968_1974-2-10_...,0
94089,data/wiki/26/503926_1976-12-31_2009.jpg,0
70321,data/imdb/68/nm0586568_rm916310528_1965-11-22_...,1
133356,data/wiki/22/3092322_1960-06-08_2015.jpg,0
22688,data/imdb/06/nm1769806_rm3182790912_1978-10-11...,1


In [24]:
#train and validation data generators

img_dim=128


train_datagen=ImageDataGenerator(rescale=1./255.,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_generator=train_datagen.flow_from_dataframe(
dataframe=train,
directory=None,
x_col='full_path',
y_col='gender',
#subset="training",
#try a smaller batch size                                            
batch_size=32,
seed=42,
shuffle=True,
class_mode='binary', 
target_size=(img_dim,img_dim))

Found 125534 validated image filenames belonging to 2 classes.


In [23]:
#test data generator

test_datagen=ImageDataGenerator(rescale=1./255.)

#valid_datagen=ImageDataGenerator(rescale=1./255.)
                                 
valid_generator=test_datagen.flow_from_dataframe(
dataframe=valid,
directory=None,
x_col='full_path',
y_col='gender',
#subset="validation",
batch_size=32, #***Set this to some number that divides your total number of images in your test set exactly
seed=42,
shuffle=True,
class_mode='binary',
target_size=(img_dim,img_dim))


test_generator=test_datagen.flow_from_dataframe(
dataframe=testdf,
directory=None,
x_col='full_path',
y_col=None,
batch_size=1, #***Set this to some number that divides your total number of images in your test set exactly
seed=42,
shuffle=False,
class_mode=None,
target_size=(img_dim,img_dim))




Found 31384 validated image filenames belonging to 2 classes.
Found 52566 validated image filenames.


In [25]:
#Define model

inputs = Input(shape = (img_dim,img_dim, 3))

# Begin
model = Conv2D(filters=32, kernel_size=(3, 3), padding='same', strides= 2, activation='relu', input_shape=(img_dim, img_dim, 3))(inputs)
model = MaxPool2D(pool_size=(3, 3), strides= 2)(model)
model = BatchNormalization(momentum=0.15)(model)

model = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(model)
model = BatchNormalization(momentum=0.15)(model)
#model = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(model)
#model = BatchNormalization(momentum=0.15)(model)

model = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu')(model)
model=Flatten()(model)

gender_model = Dense(512, activation='relu')(model)
gender_model = Dropout(0.4)(gender_model)
gender_model = Dense(1, activation='sigmoid',name='gender')(gender_model)

#gender_model.summary()
#model.compile(optimizer=tf.keras.optimizers.Adam(0.002), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])



In [26]:
model = Model(inputs=inputs, outputs=gender_model)
model.compile(optimizer = tf.keras.optimizers.Adam(0.002), loss ='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 31, 31, 32)        128       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 31, 31, 64)        18496     
_________________________________________________________________
batch_normalization_4 (Batch (None, 31, 31, 64)        256       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 31, 31, 128)       7385

In [27]:
callbacks = [tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss',restore_best_weights=True)]

BATCH_SIZE=128

#steps_per_epoch = TotalTrainingSamples / TrainingBatchSize
#validation_steps = TotalvalidationSamples / ValidationBatchSize

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

#for one output:
history = model.fit_generator(generator=train_generator,steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,validation_steps=STEP_SIZE_VALID,
                    epochs=5,callbacks=callbacks)



Epoch 1/5
 103/3922 [..............................] - ETA: 46:13 - loss: 8.1961 - accuracy: 0.5740

/Users/varuni/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:786: UserWarning: Possibly corrupt EXIF data.  Expecting to read 206 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag)
/Users/varuni/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:786: UserWarning: Possibly corrupt EXIF data.  Expecting to read 724 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag)
/Users/varuni/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:786: UserWarning: Possibly corrupt EXIF data.  Expecting to read 65536 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag)
/Users/varuni/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:786: UserWarning: Possibly corrupt EXIF data.  Expecting to read 170 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag)
/Users/varuni/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:786: UserWarnin

 960/3922 [======>.......................] - ETA: 35:59 - loss: 1.4328 - accuracy: 0.6565

/Users/varuni/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:804: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 0. 
  warnings.warn(str(msg))


1028/3922 [======>.......................] - ETA: 34:59 - loss: 1.3812 - accuracy: 0.6568

...

/Users/varuni/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:786: UserWarning: Possibly corrupt EXIF data.  Expecting to read 278 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag)
/Users/varuni/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:804: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 8. 
  warnings.warn(str(msg))


1605/3922 [===========>..................] - ETA: 27:11 - loss: 1.1137 - accuracy: 0.6492

/Users/varuni/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:804: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


2420/3922 [=================>............] - ETA: 17:43 - loss: 0.9477 - accuracy: 0.6502

KeyboardInterrupt: 

In [ ]:
model.save("cnn_gender_model.h5")
print("Saved model to disk")

In [ ]:
#Plot gender history

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('gender model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('gender_only_loss.png')
plt.clf()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('gender model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('gender_only_accuracy.png')
plt.clf()

In [15]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,val_loss,val_accuracy,loss,accuracy,epoch
0,15.424949,0.504222,7.727757,0.498634,0
1,-0.000000,0.504222,7.728021,0.498976,1
2,15.424949,0.504222,7.729126,0.498896,2
3,15.424949,0.504222,7.729123,0.498904,3
4,-0.000000,0.504222,7.729004,0.498896,4


In [1]:
#Evaluate
#loss, accuracy = model.evaluate_generator(generator=valid_generator,steps=STEP_SIZE_VALID)

#print("Test accuracy:".format(accuracy))

In [17]:
#Make predictions

test_generator.reset() #reset the test_generator before whenever you call the predict_generator
gender_predictions=model.predict_generator(generator=test_generator,steps=STEP_SIZE_TEST,verbose=1)

 2410/52566 [>.............................] - ETA: 8:07

/Users/varuni/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:804: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 3. 
  warnings.warn(str(msg))


11225/52566 [=====>........................] - ETA: 6:40

/Users/varuni/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:786: UserWarning: Possibly corrupt EXIF data.  Expecting to read 16036032132 bytes but only got 0. Skipping tag 50687
  " Skipping tag %s" % (size, len(data), tag)


34096/52566 [==================>...........] - ETA: 2:58

/Users/varuni/anaconda3/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:804: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


52566/52566 [==============================] - 506s 10ms/step


In [18]:
test_predictions = gender_predictions.flatten()

In [19]:
gender_results=[]
for item in test_predictions:
    if item<=0.5:
        gender_results.append('0')
    else:
        gender_results.append('1')
        

In [2]:
print(gender_results)

In [20]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

#  predicted_label = np.argmax(predictions_array)
  if predictions_array == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("Predicted: {}, Actual: ({})".format(predictions_array,true_label),color=color)


In [3]:
i=0
plot_image(i, gender_results, test_labels, test_images)